In [68]:
# importing libraries

import numpy as np
import pandas as pd
import psycopg2 as pg2
import nltk
import re
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

In [59]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.width  = 1000

In [34]:
# defining functions

def getDataFromDB(query):
    
    try:
        conn = pg2.connect(dbname='twitterDB',user='postgres',password='postgres',host='localhost',port='5432')
        
        if conn.closed ==0:  # checks the system connection is active
            print("Connected to the DB")
            
            cur = conn.cursor()
            
            cur.execute(query)
            records = cur.fetchall()
            
            print("Data fetched")
            
            return records
            
    except Exception as e:
        print("Error thrown")
        print(e)
    
    finally:
        print("Closing connection")
        cur.close()
        conn.close()
        
    

In [62]:
def getDataFrame(val):
    df = pd.DataFrame(val,columns=['index','username','time_created','tweet','retweet_count','place_tweet','location_user'])
    return df    

In [64]:
query1 = 'select * from tweets'

In [65]:
val = getDataFromDB(query1)

Connected to the DB
Data fetched
Closing connection


In [66]:
df = getDataFrame(val)

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147701 entries, 0 to 147700
Data columns (total 7 columns):
index            147701 non-null int64
username         147701 non-null object
time_created     147701 non-null object
tweet            147701 non-null object
retweet_count    147701 non-null int64
place_tweet      102521 non-null object
location_user    2349 non-null object
dtypes: int64(2), object(5)
memory usage: 7.9+ MB


In [42]:
df.describe()

,index,retweet_count
count,147701.000000,147701.0
mean,73865.630524,0.0
std,42640.927163,0.0
min,3.000000,0.0
25%,36940.000000,0.0
50%,73868.000000,0.0
75%,110793.000000,0.0
max,147718.000000,0.0


In [38]:
df.head()

,index,username,time_created,tweet,retweet_count,place_tweet,location_user
0,3,lionslide,2019-01-15 19:12:47+05:30,RT @iKNOMOMMY: LOOK AT LOVE SCENARIO SITTING T...,0,None,None
1,4,octopusjokes,2019-01-15 19:12:47+05:30,"Take a bow, Ms. Channing",0,the library,None
2,5,hawley_lisa,2019-01-15 19:12:47+05:30,RT @ScottMStedman: It would be a shame if this...,0,Germany (Not Really),None
3,6,SoniaSofia80,2019-01-15 19:12:47+05:30,RT @m_yosry2012: Coming baby 😂 https://t.co/Rx...,0,North,None
4,7,dominicdreams,2019-01-15 19:12:47+05:30,@ShadowhuntersTV Why they can be like this all...,0,spain,None


In [45]:
# Dropping retweet and index column

df = df.drop(['retweet_count','index'],axis =1)

In [46]:
df.head()

,username,time_created,tweet,place_tweet,location_user
0,lionslide,2019-01-15 19:12:47+05:30,RT @iKNOMOMMY: LOOK AT LOVE SCENARIO SITTING T...,None,None
1,octopusjokes,2019-01-15 19:12:47+05:30,"Take a bow, Ms. Channing",the library,None
2,hawley_lisa,2019-01-15 19:12:47+05:30,RT @ScottMStedman: It would be a shame if this...,Germany (Not Really),None
3,SoniaSofia80,2019-01-15 19:12:47+05:30,RT @m_yosry2012: Coming baby 😂 https://t.co/Rx...,North,None
4,dominicdreams,2019-01-15 19:12:47+05:30,@ShadowhuntersTV Why they can be like this all...,spain,None


In [50]:
df['len_tweet'] = df.tweet.str.len()

In [51]:
df.head()

,username,time_created,tweet,place_tweet,location_user,len_tweet
0,lionslide,2019-01-15 19:12:47+05:30,RT @iKNOMOMMY: LOOK AT LOVE SCENARIO SITTING T...,None,None,140
1,octopusjokes,2019-01-15 19:12:47+05:30,"Take a bow, Ms. Channing",the library,None,24
2,hawley_lisa,2019-01-15 19:12:47+05:30,RT @ScottMStedman: It would be a shame if this...,Germany (Not Really),None,140
3,SoniaSofia80,2019-01-15 19:12:47+05:30,RT @m_yosry2012: Coming baby 😂 https://t.co/Rx...,North,None,54
4,dominicdreams,2019-01-15 19:12:47+05:30,@ShadowhuntersTV Why they can be like this all...,spain,None,81


In [52]:
df.describe()

,len_tweet
count,147701.000000
mean,104.409435
std,55.427917
min,4.000000
25%,64.000000
50%,103.000000
75%,140.000000
max,962.000000
